In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
import MathOptInterface as MOI
import Ipopt 
using LinearAlgebra, Plots
import FiniteDiff
import ForwardDiff as FD
using MeshCat
using Test
using Plots

include(joinpath(@__DIR__, "utils", "fmincon.jl"))

In [ ]:
include(joinpath(@__DIR__, "utils", "quadruped_new.jl"))
model = UnitreeA1()
@show num_positions(model.mech)
@show num_velocities(model.mech)
@show joints(model.mech)
@show bodies(model.mech)

In [ ]:
xic = initial_state(model)
q = xic[1:num_positions(model.mech)]

@show get_trunk_position(model, q)
@show get_foot_position(model, q, "RR")
@show get_foot_position(model, q, "FL")
@show get_trunk_velocity(model, xic)

In [ ]:
# visualize initial state
mvis = initialize_visualizer(model)
xic = initial_state(model)
set_configuration!(mvis, xic[1:num_positions(model.mech)])
render(mvis)

In [ ]:
Xref, Uref, t_vec = reference_trajectory(model)

In [ ]:
@show length(Xref)
@show length(Uref)
@show length(t_vec)

In [ ]:
qs = [xk[1:num_positions(model.mech)] for xk in Xref] # extract positions from state vector

mvis = initialize_visualizer(model)
xic = initial_state(model)
set_configuration!(mvis, xic[1:num_positions(model.mech)])
anim = MeshCat.Animation(mvis, t_vec, qs)
setanimation!(mvis, anim)
render(mvis)

In [ ]:
RR_foot_poss = [get_foot_position(model, qs[i]) for i in 1:length(qs)]
RR_foot_poss_m = hcat(RR_foot_poss...)
plot(t_vec, RR_foot_poss_m', xlabel="Time Step", ylabel="Base Height (m)", title="RR Foot Position", label=["x" "y" "z"])

In [ ]:
## set up discrete time dynamics

# our continuous time dynamics function is xdot = dynamics(model, x, u)

function hermite_simpson_ground(params::NamedTuple, x1::Vector, x2::Vector, u_hat::Vector)::Vector
    model = params.model
    u = u_hat[1:control_dim(model)]
    dt = u_hat[end]
    
    x12 = 0.5*(x1+x2) + dt/8*(dynamics(model, x1, u) - dynamics(model, x2, u))
    return x1 + dt/6 * (dynamics(model, x1, u) + 4 * dynamics(model, x12, u) + dynamics(model, x2, u)) - x2
end

In [ ]:
## set up cost

function quadracat_cost(params::NamedTuple, Z::Vector)::Real
    idx, N = params.idx, params.N
    Q, R, Qf = params.Q, params.R, params.Qf
    Xref,Uref = params.Xref, params.Uref

    J = 0
    for i=1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        J += 0.5*(xi - Xref[i])'*Q*(xi - Xref[i]) + 0.5*(ui - Uref[i])'*R*(ui - Uref[i])
    end
    xN = Z[idx.x[N]]
    J += 0.5*(xN - Xref[N])'*Qf*(xN - Xref[N])
        
    return J

end

In [ ]:
## set up equality constraints 

function quadracat_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N = params.idx, params.N

    c = zeros(eltype(Z), idx.nc)

    for i=1:(N-1)
        xk = Z[idx.x[i]]
        uk = Z[idx.u[i]]
        xk1 = Z[idx.x[i+1]]

        c[idx.c[i]] = hermite_simpson_ground(params, xk, xk1, uk)
    end
    return c 
end

function quadracat_stance_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N, model = params.idx, params.N, params.model
    nq = params.nq
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), 0)
    
    # each foot is pinned to the ground
    for i=[1:3; N-2:N]
        xk = Z[idx.x[i]]
        q = xk[1:nq]
        c = vcat(c, [
                            get_foot_position(model, q, "RR")[3]; 
                            get_foot_position(model, q, "RL")[3];
                            get_foot_position(model, q, "FL")[3];
                            get_foot_position(model, q, "FR")[3]
                            ])
    end

    return c

end

function quadracat_equality_constraint(params::NamedTuple, Z::Vector)::Vector
    N, idx, xic = params.N, params.idx, params.xic
    
    # TODO: stack up all of our equality constraints 
    
    # should be length 2*nx + (N-1)*nx + N 
    # inital condition constraint (nx)       (constraint 1)
    # terminal constraint         (nx)       (constraint 2)
    # dynamics constraints        (N-1)*nx   (constraint 3-6)
    # stance constraint           N          (constraint 7-8)
    ic_con = Z[idx.x[1]] - xic
    return [ic_con; 
    quadracat_dynamics_constraints(params, Z); 
    quadracat_stance_constraint(params, Z)
    ]
end

In [ ]:
## set up inequality constraint 

function quadracat_inequality_constraint(params::NamedTuple, Z::Vector)::Vector
    idx, N = params.idx, params.N
    model = params.model
        
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), 4*N)
    for i=1:N 
        xk = Z[idx.x[i]]
        q = xk[1:num_positions(model.mech)]
        c[4*(i-1) .+ (1:4)] = [
            get_foot_position(model, q, "FL")[3];
            get_foot_position(model, q, "FR")[3];
            get_foot_position(model, q, "RR")[3];
            get_foot_position(model, q, "RL")[3]
        ]
    end

    return c
end

In [ ]:
function solve_quadracat_crouch(;verbose=true)
    
    # problem size 
    nq = num_positions(model.mech)
    nv = num_velocities(model.mech)
    nx = nq+nv
    nu = control_dim(model)
    
    Xref, Uref, t_vec = reference_trajectory(model)
    N = length(t_vec)
    
    # LQR cost 
    Q = diagm([ones(4); 3*ones(3); ones(nq-7+6); zeros(nv-6)]) #diagm(ones(nx))
    R = 0.1*diagm(ones(nu+1))
    Qf = 10*diagm(ones(nx))
    
    # indexing 
    idx = create_idx(nx,nu+1,N)
    
    # initial and goal states 
    xic = Xref[1]
    xg = Xref[end]

    # initial guess 
    z0 = zeros(idx.nz)
    for i=1:(N-1)
        z0[idx.x[i]] = Xref[i]
        z0[idx.u[i]] = Uref[i]
    end
    z0[idx.x[N]] = Xref[N]

    # load all useful things into params 
    params = (
        model=model,
        nx=nx, 
        nu=nu, 
        nq=nq,
        nv=nv,
        Q = Q, 
        R = R, 
        Qf = Qf, 
        xic = xic, 
        xg=xg,
        N = N, 
        idx = idx,
        g = 9.81,
        Xref=Xref,
        Uref=Uref
        )
    
    # TODO: primal bounds 
    # height of body must be above ground
    x_l = -Inf*ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        x_l[idx.x[i][7]] = 0
    end
    
    # inequality constraint bounds
    c_l = zeros(4*N)
    c_u = Inf*ones(4*N)
    
    # choose diff type (try :auto, then use :finite if :auto doesn't work)
    diff_type = :auto 
#     diff_type = :finite
    
    
    Z = fmincon(quadracat_cost,quadracat_equality_constraint,quadracat_inequality_constraint,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-6, c_tol = 1e-6, max_iters = 10_000, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i][1:control_dim(model)]] for i = 1:(N-1)]
    dts = [Z[idx.x[i][end]] for i = 1:(N-1)]
    
    return X, U, dts, params 
end

In [ ]:
X, U, t_vec, dts, params_dircol = solve_quadracat_crouch(verbose=true)